In [1]:
using DataFrames, ForwardDiff

df = readtable("../data/aus/model_australia.txt", separator = ' ', header = false)

names!(df, [Symbol("C$i") for i in 1:4])

,C1,C2,C3,C4
1,1,0,0,0
2,0,1,0,0
3,0,0,1,0
4,35,0,0,0
5,69,34,35,0
6,70,71,70,30
7,0,0,0,1
8,1,0,0,0
9,0,1,0,0
10,0,0,1,0


In [2]:
function f(β::Vector, model::Int64 = 0, n::Int64 = 210)
    i = 1
    while i <= n
        model += log(f_ind(i, β))
        i += 1
    end
    return model/n
end

f (generic function with 3 methods)

In [3]:
function f_ind(i::Int64, β::Vector)
    data = convert(Array, df[i*7-3:i*7-1, :])
    choice = convert(Array, df[i*7:i*7, :])
    return utility(data, choice, β)
end

f_ind (generic function with 1 method)

In [4]:
function utility(data::Matrix, choice::Matrix, β::Vector, d::Int64 = 0)
    id = find(choice .== 1)
    alt = find(choice .== 0)
    c = exp(dot(vec(data[:, id]), β))
    for j in 1:length(alt)
        d += exp(dot(vec(data[:, alt[j]]), β))
    end
    return c/(c+d)
end

utility (generic function with 2 methods)

In [5]:
function g(β::Vector)
    return ForwardDiff.gradient(f, β)
end

g (generic function with 1 method)

In [6]:
function g_ind(i::Int64, β::Vector)
    return ForwardDiff.gradient(f_ind(i, Β), β)
end

g_ind (generic function with 1 method)

In [7]:
function g!(β::Vector, storage::Vector)
    s = g(β)
    storage[1:length(s)] = s[1:length(s)]
end

g! (generic function with 1 method)

In [8]:
function g_ind!(i::Int64, β::Vector, storage::Vector)
    s = g_ind(i, β)
    storage[1:length(s)] = s[1:length(s)]
end

g_ind! (generic function with 1 method)

In [9]:
function H(β::Vector)
    return ForwardDiff.hessian(f, β)
end

H (generic function with 1 method)

In [10]:
function H_ind(i::Int64, β::Vector)
    return ForwardDiff.hessian(f_ind(i, Β), β)
end

H_ind (generic function with 1 method)

In [11]:
function H!(β::Vector, storage::Matrix)
    s = H(β)
    n, m = size(s)
    storage[1:n, 1:m] = s[1:length(s)]
end

H! (generic function with 1 method)

In [12]:
function H_ind!(i::Int64, β::Vector, storage::Matrix)
    s = H_ind(i, β)
    n, m = size(s)
    storage[1:n, 1:m] = s[1:length(s)]
end

H_ind! (generic function with 1 method)

In [13]:
function newton(f::Function, g::Function, h::Function, β0::Vector,
        k::Int64 = 1, δ::Float64 = 1e-6, nmax::Int64 = 1000)
    β = β0
    n = length(β)
    δ *= δ
    H = eye(n)
    dfβ = ones(n)
    g(β, dfβ)
    while (dot(dfβ, dfβ) > δ && k <= nmax)
        g(β, dfβ)
        h(β, H)
        β -= H\dfβ
        k += 1
    end
    β, k
end

newton (generic function with 4 methods)

In [14]:
sol = newton(f, g!, H!, [0, 0, 0])

sol

([0.0283255, -0.0257532, -0.00362244], 6)